In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

!pip install -q transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [ ]:
import os
import json
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# Caminho base do Drive
base_path = "/content/drive/MyDrive/AKCIT - NLP (Desidentificação e identificação de dados sensíveis)/modelos-selecionados"

# === Carregar modelo BERTimbau salvo localmente ===

bert_model_dir = os.path.join(base_path, "ner_model")  # deve conter config.json, tokenizer_config.json, etc.
tokenizer = AutoTokenizer.from_pretrained(bert_model_dir)
model_bert = AutoModelForTokenClassification.from_pretrained(bert_model_dir)

# Criar pipeline de inferência
bert_pipeline = pipeline(
    "ner",
    model=model_bert,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

# === Carregar modelo LSTM salvo ===
lstm_model_path = os.path.join(base_path, "avaliacao_lstm", "modelo_ner_lstm_softmax.h5")
model = load_model(lstm_model_path)

# === Carregar mapeamentos do LSTM ===
def carregar_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

word2idx = carregar_json(os.path.join(base_path, "avaliacao_lstm", "word2idx.json"))
idx2tag = carregar_json(os.path.join(base_path, "avaliacao_lstm", "idx2tag.json"))


Device set to use cpu


In [ ]:

def inferencia_lstm(texto, model, word2idx, idx2tag):
    tokens = texto.lower().split()
    tokens_idx = [word2idx.get(token, word2idx["ENDPAD"]) for token in tokens]
    tokens_padded = pad_sequences([tokens_idx], maxlen=512, padding="post", value=word2idx["ENDPAD"])
    pred = model.predict(tokens_padded, verbose=0)[0]
    tags = [idx2tag[str(np.argmax(p))] for p in pred[:len(tokens)]]
    return list(zip(tokens, tags))


In [ ]:

novos_textos = [
    "Na data de 15 de março de 2023, MARIA CLARA LIMA DE SOUSA foi abordada por dois indivíduos armados na AVENIDA PRESIDENTE VARGAS, em frente ao número 1250. Eles levaram seu celular, documentos pessoais e a motocicleta HONDA/BIZ 125 ES, PLACA QET9H51.",

    "O funcionário da empresa ELETROPARÁ relatou que, por volta das 10h30, foi impedido de acessar a residência localizada na TRAVESSA JOSÉ MALCHER, BAIRRO BATISTA CAMPOS, mesmo apresentando sua identificação funcional e autorização da CELPA.",

    "Durante uma operação policial na RUA DOS TAMOIOS, bairro COQUEIRO, foi localizado o veículo FIAT/UNO MILLE, CHASSI 9BD158225V5019823, PLACA NXB-4563, que havia sido furtado no dia anterior. O veículo foi encaminhado para o pátio da SEDETRAN.",

    "A senhora ANTONIA VIEIRA informou que foi vítima de um golpe por meio de ligação telefônica onde o suposto atendente do banco BRADESCO solicitou seus dados bancários, CPF 231.456.789-00 e senha, o que resultou na transferência de R$ 4.200,00.",

    "PEDRO HENRIQUE LOPES, residente na RUA DOS CARAJÁS, nº 78, BAIRRO GUAMÁ, registrou ocorrência informando que teve sua CNH número 01234567890, além de seu RG e cartão do BANCO DO BRASIL, furtados durante o trajeto entre sua residência e o trabalho.",

    "O indivíduo ERIK OLIVEIRA DA COSTA foi detido após ser flagrado utilizando documento falso em nome de 'FELIPE ARAÚJO', com número de CPF 987.654.321-00, no momento em que tentava abrir uma conta na agência 1234-5 do ITAÚ UNIBANCO.",

    "A equipe de vigilância noturna da empresa SEGURPARA acionou a PM após verificar movimentação suspeita no galpão da empresa HYPERMAR DISTRIBUIDORA. A ocorrência foi registrada na RUA BELÉM, número 422, bairro JURUNAS.",

    "A vítima relatou que recebeu e-mails suspeitos com remetente 'atendimento@serasa.net' solicitando atualização de dados, incluindo CPF, RG e número da conta bancária do BANPARÁ. A denúncia foi registrada e os dados encaminhados à divisão de crimes cibernéticos.",

    "Foi registrado boletim de ocorrência relatando que o automóvel CHEVROLET/ONIX LT, cor preta, PLACA QDY-1123, CHASSI 9BGKS19F0JB156728, havia sido levado da garagem do condomínio RESIDENCIAL VITÓRIA por volta das 03h45 da madrugada.",

    "O senhor FÁBIO VASCONCELOS compareceu à delegacia para comunicar o desaparecimento de sua filha menor, de nome LAURA VASCONCELOS, 14 anos, que teria saído da escola às 17h30 e não retornou para casa. A escola é localizada na AV. DUQUE DE CAXIAS, nº 902."
]


benchmark_resultados = []

for i, texto in enumerate(novos_textos):
    print(f"\n📄 Texto {i+1}: {texto}\n")

    # --- BERT ---
    print("🧠 BERTimbau:")
    resultado_bert = bert_pipeline(texto)
    if resultado_bert:
        for ent in resultado_bert:
            print(f"  • {ent['word']} - {ent['entity_group']} ({ent['score']:.2f})")
    else:
        print("  • Nenhuma entidade identificada")

    # --- LSTM ---
    print("🧬 LSTM:")
    resultado_lstm = inferencia_lstm(texto, model, word2idx, idx2tag)
    entidades_lstm = [f"{tok} - {tag}" for tok, tag in resultado_lstm if tag != "O"]
    if entidades_lstm:
        for ent in entidades_lstm:
            print(f"  • {ent}")
    else:
        print("  • Nenhuma entidade identificada")

    benchmark_resultados.append({
        "texto": texto,
        "bert": resultado_bert,
        "lstm": resultado_lstm
    })


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



📄 Texto 1: Na data de 15 de março de 2023, MARIA CLARA LIMA DE SOUSA foi abordada por dois indivíduos armados na AVENIDA PRESIDENTE VARGAS, em frente ao número 1250. Eles levaram seu celular, documentos pessoais e a motocicleta HONDA/BIZ 125 ES, PLACA QET9H51.

🧠 BERTimbau:
  • MARIA CLARA LIMA DE SOUSA - PESSOA (0.99)
  • AVENIDA PRESIDENTE VARGAS - ENDEREÇO (0.96)
  • PLACA QET9H51 - VEÍCULO (0.99)
🧬 LSTM:
  • maria - B-PESSOA
  • clara - I-PESSOA
  • lima - I-PESSOA
  • de - I-PESSOA
  • sousa - I-PESSOA

📄 Texto 2: O funcionário da empresa ELETROPARÁ relatou que, por volta das 10h30, foi impedido de acessar a residência localizada na TRAVESSA JOSÉ MALCHER, BAIRRO BATISTA CAMPOS, mesmo apresentando sua identificação funcional e autorização da CELPA.

🧠 BERTimbau:
  • empresa ELETROPARÁ - EMPRESA (1.00)
  • TRAVESSA JOSÉ MALCHER - ENDEREÇO (0.98)
  • BAIRRO BATISTA CAMPOS - ENDEREÇO (0.98)
🧬 LSTM:
  • batista - I-ENDEREÇO

📄 Texto 3: Durante uma operação policial na RUA DOS TAMOIOS,

In [ ]:
import re
import unicodedata
from collections import defaultdict

import re
import unicodedata
from collections import defaultdict

def normalizar_texto(texto):
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = re.sub(r'\s+', '', texto)
    texto = re.sub(r'[^\w]', '', texto)
    return texto.lower()

def encontrar_ocorrencia_expandida(texto, entidade):
    entidade_norm = normalizar_texto(entidade)
    texto_limpo = texto
    texto_norm = normalizar_texto(texto)

    idx = texto_norm.find(entidade_norm)
    if idx == -1:
        return None

    # Mapeia a posição no texto original com base no índice normalizado
    # Reconstrói a posição no texto original aproximando por comprimento
    palavras_entidade = entidade.split()
    if not palavras_entidade:
        return None

    trecho_ideal = entidade
    padrao = re.compile(r'\b' + re.escape(palavras_entidade[0]) + r'\b.*?\b' + re.escape(palavras_entidade[-1]) + r'\b', flags=re.IGNORECASE)

    # Busca a versão mais próxima no texto original
    for match in padrao.finditer(texto):
        trecho = match.group()
        if normalizar_texto(trecho) == entidade_norm:
            return trecho

    # Fallback se não encontrou exatamente
    return entidade

def pseudonimizar_entidades(texto, entidades, estrategia="pseudonimizacao"):
    substituicoes = {}
    contadores = defaultdict(int)
    texto_anon = texto

    for ent in entidades:
        entidade_raw = ent['word'].strip()
        tipo = ent['entity_group'].upper() if 'entity_group' in ent else ent['tag'].replace('B-', '').replace('I-', '')

        contadores[tipo] += 1
        pseudonimo = f"[{tipo}_{contadores[tipo]}]" if estrategia == "pseudonimizacao" else f"[DADO_{tipo}]"

        trecho_real = encontrar_ocorrencia_expandida(texto, entidade_raw)
        if trecho_real and trecho_real not in substituicoes:
            substituicoes[trecho_real] = pseudonimo

    # Ordena substituições da maior para menor para evitar substituições parciais
    for trecho, pseudonimo in sorted(substituicoes.items(), key=lambda x: -len(x[0])):
        texto_anon = re.sub(re.escape(trecho), pseudonimo, texto_anon, flags=re.IGNORECASE)

    return texto_anon, substituicoes

# --- Função auxiliar para converter LSTM resultado em formato similar ao do BERT ---
def formatar_entidades_lstm(resultado_lstm):
    entidades_formatadas = []
    entidade_atual = []
    tag_atual = None

    for token, tag in resultado_lstm:
        if tag.startswith("B-"):
            if entidade_atual:
                entidades_formatadas.append({"word": " ".join(entidade_atual), "entity_group": tag_atual})
                entidade_atual = []
            entidade_atual = [token]
            tag_atual = tag.replace("B-", "")
        elif tag.startswith("I-") and tag_atual:
            entidade_atual.append(token)
        else:
            if entidade_atual:
                entidades_formatadas.append({"word": " ".join(entidade_atual), "entity_group": tag_atual})
                entidade_atual = []
                tag_atual = None
    if entidade_atual:
        entidades_formatadas.append({"word": " ".join(entidade_atual), "entity_group": tag_atual})

    return entidades_formatadas

# --- Execução principal ---
benchmark_resultados = []

for i, texto in enumerate(novos_textos):
    print("=" * 90)
    print(f"\n📄 Texto {i+1} Original:\n{texto}\n")

    # --- BERT ---
    resultado_bert = bert_pipeline(texto)
    print("🧠 BERTimbau - Entidades:")
    if resultado_bert:
        for ent in resultado_bert:
            print(f"  • {ent['word']} ({ent['entity_group']})")
    else:
        print("  • Nenhuma entidade identificada")

    texto_pseudo_bert, subs_bert = pseudonimizar_entidades(texto, resultado_bert)

    # --- LSTM ---
    resultado_lstm_raw = inferencia_lstm(texto, model, word2idx, idx2tag)
    entidades_lstm = formatar_entidades_lstm(resultado_lstm_raw)
    print("\n🧬 LSTM - Entidades:")
    if entidades_lstm:
        for ent in entidades_lstm:
            print(f"  • {ent['word']} ({ent['entity_group']})")
    else:
        print("  • Nenhuma entidade identificada")

    texto_pseudo_lstm, subs_lstm = pseudonimizar_entidades(texto, entidades_lstm)

    # --- Comparação ---
    print("\n🔐 Texto Pseudonimizado com BERT:")
    print(texto_pseudo_bert)

    print("\n🔐 Texto Pseudonimizado com LSTM:")
    print(texto_pseudo_lstm)

    print('\n\n')
    print(texto)
    print(texto_pseudo_bert)
    print(texto_pseudo_lstm)


    benchmark_resultados.append({
        "texto_original": texto,
        "bert_entidades": resultado_bert,
        "lstm_entidades": entidades_lstm,
        "texto_pseudo_bert": texto_pseudo_bert,
        "texto_pseudo_lstm": texto_pseudo_lstm
    })



📄 Texto 1 Original:
Na data de 15 de março de 2023, MARIA CLARA LIMA DE SOUSA foi abordada por dois indivíduos armados na AVENIDA PRESIDENTE VARGAS, em frente ao número 1250. Eles levaram seu celular, documentos pessoais e a motocicleta HONDA/BIZ 125 ES, PLACA QET9H51.

🧠 BERTimbau - Entidades:
  • MARIA CLARA LIMA DE SOUSA (PESSOA)
  • AVENIDA PRESIDENTE VARGAS (ENDEREÇO)
  • PLACA QET9H51 (VEÍCULO)

🧬 LSTM - Entidades:
  • maria clara lima de sousa (PESSOA)

🔐 Texto Pseudonimizado com BERT:
Na data de 15 de março de 2023, [PESSOA_1] foi abordada por dois indivíduos armados na [ENDEREÇO_1], em frente ao número 1250. Eles levaram seu celular, documentos pessoais e a motocicleta HONDA/BIZ 125 ES, [VEÍCULO_1].

🔐 Texto Pseudonimizado com LSTM:
Na data de 15 de março de 2023, [PESSOA_1] foi abordada por dois indivíduos armados na AVENIDA PRESIDENTE VARGAS, em frente ao número 1250. Eles levaram seu celular, documentos pessoais e a motocicleta HONDA/BIZ 125 ES, PLACA QET9H51.



Na data d